In [1]:
import wrds
import pandas as pd
import datetime
import numpy as np
import pickle
from time import time

In [ ]:
db = wrds.Connection()

In [ ]:
q = ("select a.gvkey,a.latest,b.cshoq,b.prccq,b.mkvaltq,b.cshoq*b.prccq as market_cap,b.curcdq "
     "from "
        "(select gvkey,max(datadate) as latest "
         "from "
         "compm.fundq where datadate > '2017-01-01' " 
         "group by gvkey) a inner join "
             "(select gvkey,datadate,mkvaltq,cshoq,prccq,curcdq "
                "from compm.fundq where cshoq>0 and prccq>0 and curcdq='USD') b "
    "on a.gvkey = b.gvkey and a.latest=b.datadate "
     "order by market_cap desc "
    "limit 100")

In [ ]:
a = db.raw_sql(q)
top_2000_eq_gvkey = a['gvkey'].values.tolist()
top_2000_eq_gvkey = tuple(["'%s'"%str(i) for i in top_2000_eq_gvkey])
top_2000_eq_gvkey = ",".join(top_2000_eq_gvkey)

In [ ]:
q2 = ("select datadate,gvkey,tic,saleq,cogsq,xsgaq,oiadpq,niq,"
      "cheq, rectq, invtq, acoq, ppentq, aoq, dlcq, apq, txpq, lcoq, ltq, dlttq,cshoq "
    "from compm.fundq "
     "where gvkey in (%s) ")%top_2000_eq_gvkey
c = db.raw_sql(q2)
c.shape

In [ ]:
blnc_sheet_list = ['cheq','rectq','invtq','acoq','ppentq','aoq',
                                'dlcq','apq','txpq','lcoq','ltq','dlttq','cshoq']
mqr_list = [x + '_mqr' for x in blnc_sheet_list]

In [ ]:
income_list = ['saleq','cogsq','xsgaq','oiadpq','niq']
x = c[['gvkey','datadate'] + income_list + blnc_sheet_list]
x = x[x['gvkey'] == '006066']
x.sort_values('datadate').head()

In [ ]:
t1 = time()
%run wrds_data_processing.py
print time()-t1

In [ ]:
qq = ("select gvkey,datadate,prccm "
     "from compm.secm "
     "where gvkey='160329' ")
d = db.raw_sql(qq).sort_values('datadate')
d = d.set_index('datadate')

In [ ]:
d.loc[datetime.date(2014,3,31)]

In [ ]:
qq = ("select * "
     "from compm.secm "
     "where gvkey='001690' ")
d = db.raw_sql(qq).sort_values('datadate')
d = d.set_index('datadate')

In [ ]:
d.loc[datetime.date(2014,3,31)]

In [ ]:
d.to_pickle("prccm.pkl")

In [ ]:
q5 = ("select gvkey,datadate,split "
     "from compm.sec_split "
     "where gvkey='160329' ")
e = db.raw_sql(q5).sort_values('datadate')

In [ ]:
e= pd.read_pickle('prccm.pkl')

In [1]:
from time import time

In [2]:
t1 = time()
%run build_data.py
print time()-t1

Enter your WRDS username [lakshay]:dsanghi
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done
Shape of raw dataframe: 13979,21


153.779999971


In [4]:
y = pd.read_pickle('y.pkl')

In [9]:
y.iloc[-10::]

,gvkey,saleq,cogsq,xsgaq,oiadpq,niq,cheq,rectq,invtq,acoq,ppentq,aoq,dlcq,apq,txpq,lcoq,ltq,dlttq,cshoq
datadate,,,,,,,,,,,,,,,,,,,
2015-06-30,006066,20813.0,9326.0,6477.0,4049.0,3449.0,8760.0,27911.0,1912.0,6603.0,10702.0,56841.0,5329.0,5943.0,3639.0,20624.0,98892.0,33340.0,979.530
2015-09-30,006066,19279.0,8776.0,6015.0,3552.0,2950.0,9568.0,25113.0,1613.0,5818.0,10661.0,55875.0,7539.0,5166.0,2883.0,18144.0,95198.0,32122.0,970.110
2015-12-31,006066,22059.0,9530.0,6504.0,5035.0,4463.0,8194.0,28554.0,1551.0,4205.0,10727.0,57264.0,6461.0,6028.0,2847.0,18933.0,96071.0,33428.0,965.729
2016-03-31,006066,18685.0,8836.0,7542.0,1283.0,2014.0,14869.0,26730.0,1690.0,4334.0,10909.0,60324.0,5302.0,5302.0,2203.0,21857.0,103784.0,40255.0,959.962
2016-06-30,006066,20238.0,9287.0,6791.0,3057.0,2504.0,10617.0,26546.0,1685.0,4676.0,11092.0,63440.0,4887.0,5484.0,2275.0,20939.0,102167.0,39637.0,955.844
2016-09-30,006066,19226.0,8939.0,6126.0,3034.0,2853.0,9968.0,25196.0,1730.0,4539.0,11104.0,63069.0,6920.0,5271.0,2137.0,20119.0,98447.0,35563.0,950.855
2016-12-31,006066,21771.0,9606.0,6404.0,4634.0,4501.0,8527.0,29245.0,1553.0,4563.0,10830.0,62752.0,7513.0,6209.0,3235.0,19318.0,99078.0,34655.0,945.867
2017-03-31,006066,18155.0,9126.0,6676.0,1254.0,1750.0,10695.0,25869.0,1609.0,4715.0,10865.0,63742.0,8340.0,5324.0,2747.0,20070.0,99047.0,34441.0,939.497
2017-06-30,006066,19288.0,9206.0,6642.0,2323.0,2332.0,12295.0,26958.0,1604.0,4156.0,10903.0,64579.0,8061.0,5126.0,2779.0,20000.0,101951.0,37611.0,931.940


In [10]:
21771+19226+20238+18685

79920